#**Import Modules**

Selenium, Openpyxl, Files, Time

In [0]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {"profile.managed_default_content_settings.images": 2,
         "download_restrictions": 3}
chrome_options.add_experimental_option("prefs", prefs)
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 911kB 2.8MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [83.6 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 ht

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


In [0]:
from selenium.common.exceptions import *

In [0]:
import time

In [0]:
import openpyxl

In [0]:
from google.colab import files

#**Fetch all Volume Links**

In [0]:
MAINURLFORMAT = "https://www.sciencedirect.com/journal/journal-of-development-economics/issues?page="
volUrls = []

for i in range(3):
    # access page
    print("Reading, " + MAINURLFORMAT + str(i + 1))
    wd.get(MAINURLFORMAT + str(i + 1))
    time.sleep(0.5)

    # toggle all dropdown buttons
    btns = wd.find_elements_by_xpath("//button[@aria-expanded]")
    for btn in btns:
        if "accordion" in btn.get_attribute("id"):
            while "true" not in btn.get_attribute("aria-expanded"):
                btn.click()
                time.sleep(0.5)

    # fetch volume urls
    a_tags = wd.find_elements_by_xpath("//a[@href]")
    for tag in a_tags:
        if "vol" in tag.get_attribute("href") and not tag.get_attribute("href") in volUrls:
            volUrls.append(tag.get_attribute("href"))

Reading, https://www.sciencedirect.com/journal/journal-of-development-economics/issues?page=1
Reading, https://www.sciencedirect.com/journal/journal-of-development-economics/issues?page=2
Reading, https://www.sciencedirect.com/journal/journal-of-development-economics/issues?page=3


In [0]:
# check missing volume no.
tmps = []

for volurl in volUrls:
    tmp = volurl[volurl.index("vol") + 4:]
    tmp = tmp[:tmp.index("/")]
    if tmp not in tmps:
        tmps.append(tmp)

for i in range(144):
    if str(i + 1) not in tmps:
        print(i + 1)

#**Fetch and Write Data from each Volumes**

In [0]:
wb = openpyxl.Workbook()
ws = wb.active

In [0]:
 r = 2

 # write header
 ws.cell(row = 1, column = 1).value = 'journal'
 ws.cell(row = 1, column = 2).value = 'year'
 ws.cell(row = 1, column = 3).value = 'title'
 ws.cell(row = 1, column = 4).value = 'keywords'
 ws.cell(row = 1, column = 5).value = 'authors'

 for volurl in volUrls:
    # access volume page
    wd.get(volurl)
    time.sleep(1)

    # fetch volume no. from url
    vol = volurl[volurl.index("vol") + 4:]
    vol = vol[:vol.index("/")]

    # fetch year
    try:
        yr = wd.find_element_by_xpath("//h3[@class='js-issue-status text-s']").get_attribute("innerText").split(" ")[-1].replace(")", "").replace("(", "").replace(" ", "")
    except NoSuchElementException:
        continue
    
    print("Reading Volume from Year, " + yr + " : "  + volurl)

    # fetch article urls
    articleUrls = []
    a_tags = wd.find_elements_by_xpath("//a[@href]")
    for tag in a_tags:
        if (("article/pii" in tag.get_attribute("href")) and (not "pdf" in tag.get_attribute("href")) and (not "Editorial Board" in tag.text) and (not tag.get_attribute("href") in articleUrls)):
            articleUrls.append(tag.get_attribute("href"))

    for articleurl in articleUrls:
        # access article page
        wd.get(articleurl)
        time.sleep(0.8)
        #print("Reading article, " + articleurl)

        # fetch authors
        authors = []
        auth_gn_tags = wd.find_elements_by_xpath("//span[@class='text given-name']")
        auth_sn_tags = wd.find_elements_by_xpath("//span[@class='text surname']")
        for j in range(len(auth_gn_tags)):
            authors.append(auth_gn_tags[j].text + " " + auth_sn_tags[j].text)
        if(len(authors) == 0):
            continue
        #print(authors)

        # fetch title
        try:
            title = wd.find_element_by_class_name("title-text").text
        except NoSuchElementException:
            continue
        #print(title)

        # fetch keywords
        keywords = []
        kwd_header = wd.find_elements_by_class_name("keywords-section")
        for candid in kwd_header:
            try:
                if(candid.find_element_by_xpath("./h2").text == "Keywords"):
                    kwd_tags = candid.find_elements_by_xpath("./div")
                    for kwd in kwd_tags:
                        keywords.append(kwd.text)
                else:
                    continue
            except NoSuchElementException:
                continue
        if(len(keywords) == 0):
            continue
        #print(keywords)

        # write data to excel file
        print("Writing, " + title)
        ws.cell(row = r, column = 1).value = "Journal of Development Economics"
        ws.cell(row = r, column = 2).value = yr
        ws.cell(row = r, column = 3).value = title
        for j in range(len(keywords)):
            ws.cell(row = r + j, column = 4).value = keywords[j]
        for j in range(len(authors)):
            ws.cell(row = r + j, column = 5).value = authors[j]
        r += max([len(keywords), len(authors)])

    # backup for every ten volumes
    if(volUrls.index(volurl) % 20 == 0):
        print("Saving backup file")
        wb.save(filename = "journal_of_development_economics_backup.xlsx")
        time.sleep(20)
        #files.download("journal_of_development_economics_backup.xlsx")

Reading Volume, https://www.sciencedirect.com/journal/journal-of-development-economics/vol/144/suppl/C
Writing, Experimental identification of asymmetric information: Evidence on crop insurance in the Philippines
Writing, When financial incentives backfire: Evidence from a community health worker experiment in Uganda
Writing, Implementation and effects of India's national school-based iron supplementation program
Writing, Land titles and violent conflict in rural Mexico
Writing, Decentralized delivery of financial education: Experimental evidence from Rwanda
Writing, Trade and agricultural technology adoption: Evidence from Africa
Writing, Services development and comparative advantage in manufacturing
Writing, The Belt and Road Initiative: Reshaping economic geography in Central Asia?
Writing, Using social connections and financial incentives to solve coordination failure: A quasi-field experiment in India's manufacturing sector
Writing, Extractive resource policy and civil conflict: 

#**Save File**

In [0]:
wb.save(filename = "journal_of_development_economics.xlsx")

#**Download FIle**

In [0]:
files.download("journal_of_development_economics.xlsx")